<a href="https://colab.research.google.com/github/caglakacar/image_processing/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split

def extract_features(image):
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    digit_moments = []
    bounding_boxes = []

    for contour in contours:
        moments = cv2.moments(contour)
        digit_moments.extend(list(moments.values()))
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.extend([x, y, w, h])

    return digit_moments, bounding_boxes

digits = datasets.load_digits()

moment_values = []
bounding_boxes = []

for digit in digits.images:
    digit_uint8 = cv2.convertScaleAbs(digit)
    moments, boxes = extract_features(digit_uint8)
    moment_values.append(moments)
    bounding_boxes.append(boxes)

max_moments_length = max(len(m) for m in moment_values)
max_boxes_length = max(len(b) for b in bounding_boxes)

moment_values_padded = [m + [0] * (max_moments_length - len(m)) for m in moment_values]
bounding_boxes_padded = [b + [0] * (max_boxes_length - len(b)) for b in bounding_boxes]

moment_values_array = np.array(moment_values_padded, dtype='float32')
bounding_boxes_array = np.array(bounding_boxes_padded, dtype='float32')

features = np.hstack((moment_values_array, bounding_boxes_array))
df_data = pd.DataFrame(features)

totalItems = df_data.shape[1]

X_train, X_test, y_train, y_test = train_test_split(
    df_data, digits.target, test_size=0.2, shuffle=False
)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(totalItems,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

Epoch 1/100
45/45 [==============================] - 1s 2ms/step - loss: 79.4636 - accuracy: 0.1795
Epoch 2/100
45/45 [==============================] - 0s 2ms/step - loss: 19.8206 - accuracy: 0.4106
Epoch 3/100
45/45 [==============================] - 0s 2ms/step - loss: 13.8579 - accuracy: 0.4614
Epoch 4/100
45/45 [==============================] - 0s 2ms/step - loss: 10.6324 - accuracy: 0.5261
Epoch 5/100
45/45 [==============================] - 0s 2ms/step - loss: 7.8648 - accuracy: 0.5790
Epoch 6/100
45/45 [==============================] - 0s 2ms/step - loss: 7.4744 - accuracy: 0.5560
Epoch 7/100
45/45 [==============================] - 0s 2ms/step - loss: 7.0730 - accuracy: 0.6082
Epoch 8/100
45/45 [==============================] - 0s 2ms/step - loss: 7.3551 - accuracy: 0.5915
Epoch 9/100
45/45 [==============================] - 0s 2ms/step - loss: 5.7863 - accuracy: 0.6319
Epoch 10/100
45/45 [==============================] - 0s 2ms/step - loss: 5.3494 - accuracy: 0.6228
Epoch